In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import time
import re

import json

from skrub import TableReport
from ollama import Client
from pdfquery import PDFQuery
from pprint import pprint
from IPython.display import display, HTML

import bid_utils

# Preparation des données (dataframe) pour automatiser l'extraction des informations

In [2]:
## load the dfs built in 1_collect_files
rep_drive_ATAE =  r"C:\Users\jch_m\ATAE"
rep_data_input =  r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_input"
rep_data_output = r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output"

path_df_EBP = os.path.join(rep_data_output, "df_EBP.pkl")
df_EBP = pd.read_pickle(path_df_EBP)

path_df_consult = os.path.join(rep_data_output, "df_consult.pkl")
df_consult = pd.read_pickle(path_df_consult)

path_df_rejet = os.path.join(rep_data_output, "df_rejet.pkl")
df_rejet = pd.read_pickle(path_df_rejet)

# preparation du path pour le fichier resultat 
path_df_consult_elevated = os.path.join(rep_data_output, "df_consult_elevated.pkl")

In [3]:
# Merge df_consult and df_EBP pour sync EBP ID on the mission/files
# note: no need to sync EBP with rejet, because none of the Rejet file match an EBP entry (dans la liste des repertoires et fichiers). normal ?

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

df_consult_ebp = pd.merge(df_consult, df_EBP, on=match_columns, how='left', suffixes=('_consult', '_ebp'))
df_consult_ebp['ID EBP'] = df_consult_ebp['ID EBP_ebp'].fillna("no EBP")
df_consult_ebp = df_consult_ebp.drop(columns=['ID EBP_ebp','ID EBP_consult', 'statut_ebp'])

#df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].strip()
df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].str.strip()

In [4]:
# define a unique 'no EBP xx'  for each combinaison of SPS+Ville+Entreprise+Mission, for all related files
#
mask = df_consult_ebp["ID EBP"] == "no EBP"

# Create the combined series only for the masked rows
combined_series_for_update = (
    df_consult_ebp.loc[mask, 'SPS Name'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Ville'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Entreprise'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Mission'].astype(str)
)

# Generate unique IDs for this series
unique_ids, _ = pd.factorize(combined_series_for_update)

# Assign back to the original DataFrame using the mask
df_consult_ebp.loc[mask, "ID EBP"] = "no EBP " + (unique_ids + 1).astype(str)


In [5]:
# Add columns "AO_docs" (True/False), "AO_Doc_type"(CCTP, CCAP, RC, AAPC, Memo_tech), "Commande"(True/False)

# Set AO_doc_type
# Initialize all with default value
df_consult_ebp['AO_doc_type'] = 'no type'

# Define mask for inclusion et exclusion 
df_consult_ebp['AO_docs'] = (
    (df_consult_ebp['file_path'].str.lower().str.contains('devis') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul')) | 
    (df_consult_ebp['file_path'].str.lower().str.contains(r'\\adm') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul'))
)
mask = df_consult_ebp['AO_docs'] == True

# Création d'un masque d'exclusion : lot est retiré provisoirement
mots_cles_a_exclure = ["plan ", "assurance", "honneur", "plans", "coupe", "vue", "facade", "archi"] 
exclusion_mask = df_consult_ebp.loc[mask, 'file_path'].str.lower().apply(
    lambda x: any(mot in x for mot in mots_cles_a_exclure)
)

# Sélection des lignes à inclure (qui ne contiennent pas les mots-clés à exclure)
inclusion_mask = mask.copy()
inclusion_mask.loc[mask] = ~exclusion_mask

# par ordre inverse d'importance, pour que les derniers checks ecrasent eventuellement les premiers
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('sps', case=False, na=False), 'AO_doc_type'] = 'CCTP'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('PA-', case=False, na=False), 'AO_doc_type'] = 'Procedure Adaptee'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('RC', case=False, na=False), 'AO_doc_type'] = 'Reglement'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('lettre', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('courier', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('consult', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('AAPC', case=False, na=False), 'AO_doc_type'] = 'AAPC' #Achat avec concurence
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('glement', case=False, na=False), 'AO_doc_type'] = 'Reglement'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('planning', case=False, na=False), 'AO_doc_type'] = 'Planning'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('phasag', case=False, na=False), 'AO_doc_type'] = 'Planning'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('programme', case=False, na=False), 'AO_doc_type'] = 'CCTP' # Technique
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCTP', case=False, na=False),'AO_doc_type'] = 'CCTP' # Technique
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCP', case=False, na=False), 'AO_doc_type'] = 'CCP' # General , Technique et Admin
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCAP', case=False, na=False), 'AO_doc_type'] = 'CCAP' #Administratif
#df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('DCE', case=False, na=False), 'AO_doc_type'] = 'CCTP' # Dossier Consult 


In [6]:
print("nb total de fichiers:",len(df_consult_ebp['AO_doc_type']))
print("nb total de fichiers de consultation:",len(df_consult_ebp[df_consult_ebp['AO_docs']==True] ))
print("nb de fichiers AO_docs avec un type identifié:",len(df_consult_ebp['AO_doc_type']) - len(df_consult_ebp[df_consult_ebp['AO_doc_type']=='no type']))
print("\nDétails par type", df_consult_ebp['AO_doc_type'].value_counts())


nb total de fichiers: 158545
nb total de fichiers de consultation: 3106
nb de fichiers AO_docs avec un type identifié: 829

Détails par type AO_doc_type
no type              157716
CCTP                    345
Reglement               244
CCAP                    181
AAPC                     28
Lettre Consult           12
Planning                  9
Procedure Adaptee         7
CCP                       3
Name: count, dtype: int64


In [7]:
# Liste de fichiers de consultations sans type clairement identifié (ou volontairement exclus)
#
list_file_AO_notype = ""
mask2 = (df_consult_ebp['AO_docs'] == True) &  (df_consult_ebp['AO_doc_type'] == 'no type')
list_file_AO_notype = [df_consult_ebp.loc[mask2, 'file_name'], df_consult_ebp.loc[mask2, 'file_path']]
for index, row in df_consult_ebp.loc[mask2, ['file_name', 'file_path']].iterrows():
    file = row['file_name']
    file_path = row['file_path'].strip()

    if any(mot in file.lower() for mot in mots_cles_a_exclure):
        pass

    bid_utils.path_to_link(file_path)


In [8]:
TableReport(df_consult_ebp)

Processing column  10 / 10


,,,,,,,,,,


In [9]:
# Creation d'un DF_consult_elevated avec une ligne par "EBP ID", 
# et l'ajout des features resultants de l'extraction 

df_consult_ao = df_consult_ebp[df_consult_ebp['AO_docs']==True]
df_consult_elevated = df_consult_ao.drop_duplicates(subset=["ID EBP"], keep="first")

# Suppression des colonnes desormais inutiles
df_consult_elevated = df_consult_elevated.drop(columns=['file_path','file_name','AO_docs','AO_doc_type'])


In [10]:
df_consult_elevated.info()


<class 'pandas.core.frame.DataFrame'>
Index: 926 entries, 0 to 158400
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SPS Name        926 non-null    object
 1   Ville           926 non-null    object
 2   Entreprise      926 non-null    object
 3   Mission         926 non-null    object
 4   statut_consult  926 non-null    object
 5   ID EBP          926 non-null    object
dtypes: object(6)
memory usage: 50.6+ KB


# Definition des prompts et functions d'extractions

In [11]:
# Prompts pour la collection des infos par type de document
#
prompt_role = "your are an assistant to analyse the bids for Coordination SPS, extract key informations followins the specified instructions in term of format and content"

###### ---- CCTP -------------- ###################
format_json_cctp = {
    "Nom Chantier": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Maitre oeuvre": "",
    "Type de Travaux": "",
    "Planning phase conception": "",
    "Planning phase realisation": "",
    "Prix des travaux": 0,
    "Duree des travaux": 0,
    "Categorie operation SPS": "",
} 
json_string_cctp = json.dumps(format_json_cctp, ensure_ascii=False, indent=2)

prompt_resumer_cctp = """
En Francais, extraire du **Texte CCxP** les **Informations** suivantes : 

**Informations**:
'Nom Chantier': scope du projet, objet du chantier, objectif du programme, 
'Lieu du Chantier': ville  Commune  Departement Rue , 
'Maitre ouvrage': nom du Maitre d'ouvrage du projet,
'Maitre oeuvre': nom du maitre d'oeuvre du chantier ,
'Type de Travaux': type et nature du travaux du chantier exemple: amenagement construction ,
'Planning phase conception': date et duree Previsionnelle, 
'Planning phase realisation': date et duree Previsionnelle, 
'Prix des travaux' : en euros HT <integer>, 
'Durée Prévisionnelle des Travaux': en nombre de mois <integer>,
'Categorie operation SPS':  I II ou III, 

**Instructions**: 
Donne uniquement les informations disponibles dans le texte CCxP, sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 900 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte CCxP**: 
"""

prompt_json_cctp = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: \n{json_string_cctp}\

**Text**: 
"""

###### ---- Reglement -------------- ###################
format_json_regl = {
    "Critere Prix": 0,
    "Critere Technique": {
        "global": 0,
        "details": {
            "Moyen Humain et Experience": 0,
            "Methodologie": 0,
            "Cohérence du temps": 0,
            "Compréhension des enjeux": 0,
        }
    },
    "Prix des travaux": 0,
    "Duree des travaux": 0,
}
string_json_regl = json.dumps(format_json_regl, ensure_ascii=False, indent=2)

prompt_resumer_regl = """ 
Extract from **Texte** the following informations related to 'evaluation et critère de l'offre'  : 

**Informations**:
'Critère Prix': quel poid ou ponderation pour le critère de l'examen du prix des prestations , valeur de la note maximale ? <integer>,
'Critère Technique' : quel poids ou ponderations pour le critère de l'examen la valeur technique de l'offre ? et detaillant si disponible les sous-critères suivants <integer>,
'Moyen Humain et Experience': quel poids pour la compétences, moyen matériel ? <integer>,
'Methodologie': quel poids ou note pour ce sous-critère de la methode technique ? <integer>,
'Cohérence du temps': quel poids ou note pour ce sous-critère du temps de travail estimé pour ce service ? <integer>,
'Compréhension des enjeux': quel poids ou note pour ce sous-critère Compréhension des enjeux pour ce service ? <integer>,, 
'Prix des travaux' : en euros HT <integer>, 
'Durée Prévisionnelle des Travaux': en nombre de mois <integer>,

**Instructions**: 
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 500 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte**: 
"""

prompt_json_regl = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: 
{string_json_regl}

**Text**: \n 
""" 

###### ---- AAPC -------------- ###################
format_json_aapc = {
    "Mission": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Lot": "",
    "Tranche": "",
    "Prix des travaux": 0,
    "Duree des travaux": 0,
}
string_json_aapc = json.dumps(format_json_aapc, ensure_ascii=False, indent=2)

prompt_resumer_aapc = """ 
Extract from **Texte** the following informations related to 'evaluation et critère de l'offre'  : 

**Informations**:
'Mission': scope du projet, objet du chantier, objectif du programme, 
'Lieu du Chantier': ville  Commune  Departement Rue , 
'Maitre ouvrage': nom du Maitre d'ouvrage du projet,
'Tranche': le projet est il decoupé en plusieurs Tranche ou phases ex '2 tranches', 'non'
'Lot': le projet est il decoupé en plusieurs lots, ex '3 lots', 'non',
'Prix des travaux' : en euros HT, <integer>,
'Durée des Travaux': en nombre de mois, <integer>,

**Instructions**: 
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 500 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte**: 
"""

prompt_json_aapc = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: 
{string_json_aapc}

**Text**: \n 
""" 

###### ---- CCAP -------------- ###################
format_json_ccap= {
    "Objet du marché": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Maitre oeuvre": "",
    "tranche": "",
    "Lot": "",
}
string_json_ccap = json.dumps(format_json_ccap, ensure_ascii=False, indent=2)

prompt_resumer_ccap = """ 
Extract from **Texte** the following informations related to 'evaluation et critère de l'offre'  : 

**Informations**:
'Objet du marché': scope du projet, objet du chantier, objectif du programme, 
'Lieu du Chantier': ville  Commune  Departement Rue , 
'Maitre ouvrage': nom du Maitre d'ouvrage du projet,
'Maitre oeuvre': nom du Maitre d'oeuvre du projet, en phase de conception et de realisation 
'Tranche': le projet est il decoupé en plusieurs tranches, ex '2 tranches', 'non',
'Lot': le projet est il decoupé en plusieurs lot, ex '3 lots', 'non',

**Instructions**: 
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 500 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte**: 
"""

prompt_json_ccap = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: 
{string_json_ccap}

**Text**: \n 
""" 


In [12]:
# Init Ollama, with list of model, prompt

ollama_url = "http://localhost:11434"

# Get an ollama client
llmclient = Client(host=ollama_url)
model_options = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0.05,
    "top_p": 0.2,
    "num_ctx": 9000 # max number of tokens to input
}
"""
model_options_json = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0.05,
    "top_p": 0.2,
    "format": "json",
    "num_ctx": 9000 # max number of tokens to input
}
"""

# 'mistral-small3.1=14G  llama3.2:latest=2G gemma3:4b=3.3G
#list_model = ["gemma3:4b", "llama3.2", "minicpm-v", "mistral-small3.1"]
list_model = ["gemma3:12b", "gemma2:9b", "llama3.1:8b", "mistral:7b"]



In [13]:
## Run all models for the same task (prompt + file) for comparison
##

def run_model_CCTP(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_cctp + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")
    
    #JSON
    prompt_full_j = prompt_json_cctp + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)


def run_model_reglement(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_regl + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")

    #JSON
    prompt_full_j = prompt_json_regl + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)


def run_model_aapc(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_aapc + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")

    #JSON
    prompt_full_j = prompt_json_aapc + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)


def run_model_ccap(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_ccap + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")
    
    #JSON
    prompt_full_j = prompt_json_ccap + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)




In [14]:
# Teste plusieurs models sur l'enchaienement d'analyse de fichiers pour 1 EBP ID
def test_models_for_1_ebp(list_model_to_test):
    ebp_id_l = ["no EBP 13"] # impose ce projet "no EBP 13" => CCTP,CCAP, Regl  / #no EBP 9717
    ebp_id = ebp_id_l[0]
    list_consult_type = ['Lettre Consult', 'CCP', 'CCTP', 'CCAP', 'Planning', 'AAPC', 'Reglement']

    #filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == selected_ebp_id[0]].iloc[0]
    #print(f"\n ***** ID:{selected_ebp_id} statut_consult: {filtered_row['statut_consult']} ****** ")
    filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
    print(f"\n ***** ID:{ebp_id} statut_consult: {filtered_row['statut_consult']} ***** ")
    print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
    
    for model_name in list_model_to_test:
        print(f" ===================== {model_name} ==========================")
        for doc_type in list_consult_type:
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == doc_type)]['file_path']
        
            if not list_file_consult.empty:
                print("Doc type:",doc_type)
                text_multi_files = ""
                for file_path in list_file_consult:
                    #nb_segment = file_path.split("\\")
                    #print("==> ", nb_segment[-1].strip())
                    bid_utils.path_to_link(file_path.strip(), option=None)
                    text_file = bid_utils.loadpdf_as_text(file_path.strip())
                    text_multi_files += text_file + "\n" # Contatenate text
                
                # Trim text to suppress long' '
                text_input = re.sub(r'\s+', ' ', text_multi_files).strip()
                
                # Run model for each doc type
                if doc_type == "CCTP" or doc_type == "CCP" or doc_type == "Lettre Consult" or doc_type == "Planning" :
                    json_result = run_model_CCTP(text_input, model_name)
                    #bid_utils.update_df_with_json_cctp(json_result , ebp_id, df_consult_elevated)
                    
                if doc_type == "Reglement" or doc_type == "Lettre Consult":
                    json_result = run_model_reglement(text_input, model_name)
                    #bid_utils.update_df_with_json_regl(json_result , ebp_id, df_consult_elevated)
                    
                if doc_type == "AAPC" or doc_type == "Planning":
                    json_result = run_model_aapc(text_input, model_name)
                    #bid_utils.update_df_with_json_aapc(json_result , ebp_id, df_consult_elevated)
                    
                if doc_type == "CCAP" or doc_type == "CCP" or doc_type == "Planning":
                    json_result = run_model_ccap(text_input, model_name)
                    #bid_utils.update_df_with_json_ccap(json_result , ebp_id, df_consult_elevated)
                    
#list_model = ["gemma3:4b", "gemma3:12b", "gemma2:9b", "llama3.1:8b", "llama3.2:latest", "qwen3:8b"]
list_model = ["gemma2:9b", "llama3.1:8b", "llama3.2:latest"]
#test_models_for_1_ebp(list_model)

# Main part
- boucle pour extraire les infos
- verification des infos collectées

In [22]:
#### Enchainement des de l'extraction des docs d'un projet pour cumuler les infos.

# modele fixé
model_name = "gemma3:4b" # ou "llama3.2:latest" "gemma3:4b"

# Liste ordonnées des types de doc Consult à explorer
list_consult_type = ['Lettre Consult', 'CCP', 'CCTP', 'CCAP', 'Planning', 'AAPC', 'Reglement']

consult_mask = (df_consult_ebp['AO_docs'] == True) 
#cctp_mask = df_consult_ebp['AO_doc_type'] == "CCTP"

consult_ebp_ids = df_consult_ebp.loc[consult_mask, 'ID EBP'].unique()
print(f"il y a {len(consult_ebp_ids)} projets avec au moins 1 document du type 'Consultation' ")
print("nb de fichiers AO_docs avec un type identifié:",len(df_consult_ebp['AO_doc_type']) - len(df_consult_ebp[df_consult_ebp['AO_doc_type']=='no type']))

count = 0
start_exec = 200
max_count = 500
nb_model_called = 0
count_ebp_model = 0 

for ebp_id in consult_ebp_ids:

    filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
    print(f"\n ***** {count+1}/{max_count}:  ID:{ebp_id} statut_consult: {filtered_row['statut_consult']} ***** ")
    print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")

    for doc_type in list_consult_type:
        list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == doc_type)]['file_path']
        flag_callmodel = False
        if not list_file_consult.empty:
            print("Doc type:",doc_type)
            text_multi_files = ""
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
            
            # Trim text to suppress long' '
            text_input = re.sub(r'\s+', ' ', text_multi_files).strip()

            if count >= start_exec:
                # Run model for each doc type
                flag_callmodel = True
                if doc_type == "CCTP" or doc_type == "CCP" or doc_type == "Lettre Consult" or doc_type == "Planning" :
                    json_result = run_model_CCTP(text_input, model_name)
                    bid_utils.update_df_with_json_cctp(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "Reglement" or doc_type == "Lettre Consult":
                    json_result = run_model_reglement(text_input, model_name)
                    bid_utils.update_df_with_json_regl(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "AAPC" or doc_type == "Planning":
                    json_result = run_model_aapc(text_input, model_name)
                    bid_utils.update_df_with_json_aapc(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "CCAP" or doc_type == "CCP" or doc_type == "Planning":
                    json_result = run_model_ccap(text_input, model_name)
                    bid_utils.update_df_with_json_ccap(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
    count_ebp_model += 1
    if count_ebp_model % 50 == 0:
        df_consult_elevated.to_pickle(path_df_consult_elevated)
        print(f"\nCount {count_ebp_model}: DataFrame successfully saved to '{path_df_consult_elevated}'")

    if count > max_count:
        break
    count += 1

il y a 926 projets avec au moins 1 document du type 'Consultation' 
nb de fichiers AO_docs avec un type identifié: 829

 ***** 1/500:  ID:no EBP 1 statut_consult: Perdu ***** 
Ville:BOUGUENAIS  Entreprise:BGTA Ministère environnement  Mission:Sécurisation BGTA
Doc type: Reglement



 ***** 2/500:  ID:no EBP 3 statut_consult: Devis ***** 
Ville:BOUGUENAIS  Entreprise:MAIRIE  Mission:Salle de sport Joel Dubois et Cossec
Doc type: CCTP



 ***** 3/500:  ID:no EBP 13 statut_consult: Perdu ***** 
Ville:ST NAZAIRE  Entreprise:SONADEV  Mission:CONSTRUCTION GROUPE SCOLAIRE
Doc type: CCTP


Doc type: AAPC


Doc type: Reglement


...


LLM response pour resumer: in 270s

 ***** 497/500:  ID:no EBP 7075 statut_consult: Chantier ***** 
Ville:QUEBRIAC  Entreprise:SCI HPC  Mission:Bâtiment de stockage
Doc type: CCTP



LLM response pour resumer: in 262s

 ***** 498/500:  ID:240844 statut_consult: Chantier ***** 
Ville:RENNES  Entreprise:TERRITOIRES RENNES  Mission:ZAC Maurepas Gayeulles-Démolitions
Doc type: CCTP



LLM response pour resumer: in 77s
Doc type: CCAP



LLM response pour resumer: in 271s
Doc type: Reglement



LLM response pour resumer: in 161s

 ***** 499/500:  ID:220164 statut_consult: Archive ***** 
Ville:RENNES  Entreprise:PATTON LEGENDRE  Mission:DEMOLITION

 ***** 500/500:  ID:no EBP 7686 statut_consult: Archive ***** 
Ville:PATTON_Contrat PI  Entreprise:Annexe 3  Mission:Planning général_ind2_220113.pdf

Count 500: DataFrame successfully saved to 'C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output\df_consult_elevated.pkl'

 ***** 501/500:  ID:230903 statut_consult: Archive ***** 
Ville:BRUZ  Entreprise:ESID  Mission:USINE ENERGIE Luminaire

 ***** 502/500:  ID:230912 statut_consult: Archive ***** 
Ville:CHANTEPIE  Entreprise:MAIRIE  Mission:Cimetière


In [20]:
print("nb of count_ebp_model",count_ebp_model)
print("nb of nb_model_called",nb_model_called, "\n")

#f_consult_ebp.loc[consult_mask].head()
df_consult_elevated.loc[consult_mask].info()

nb of count_ebp_model 202
nb of nb_model_called 85 

<class 'pandas.core.frame.DataFrame'>
Index: 926 entries, 0 to 158400
Data columns (total 34 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   SPS Name                             926 non-null    object 
 1   Ville                                926 non-null    object 
 2   Entreprise                           926 non-null    object 
 3   Mission                              926 non-null    object 
 4   statut_consult                       926 non-null    object 
 5   ID EBP                               926 non-null    object 
 6   regl crit-prix                       23 non-null     float64
 7   regl crit-tech global                23 non-null     object 
 8   regl duree-travaux                   23 non-null     object 
 9   regl prix travaux                    23 non-null     object 
 10  regl crit-tech-moyen_humain          23 non-nul

In [23]:
df_consult_elevated.columns

Index(['SPS Name', 'Ville', 'Entreprise', 'Mission', 'statut_consult',
       'ID EBP', 'regl crit-prix', 'regl crit-tech global',
       'regl duree-travaux', 'regl prix travaux',
       'regl crit-tech-moyen_humain', 'regl crit-tech-methodologie',
       'regl crit-tech-coherence_temps', 'cctp nom_chantier',
       'cctp type travaux', 'cctp duree travaux', 'cctp prix travaux',
       'cctp planning conception', 'cctp maitre ouvrage', 'cctp cat SPS',
       'aapc mission', 'aapc lieu', 'aapc m_ouvrage', 'aapc tranche',
       'aapc prix_travaux', 'aapc duree_travaux',
       'regl crit-tech-comprehension_enjeux', 'cctp lieu',
       'cctp maitre oeuvre', 'ccap objet', 'ccap lieu', 'ccap lot',
       'ccap m_ouvrage', 'ccap m_oeuvre', 'aapc tot', 'ccap tranche'],
      dtype='object')

In [24]:
# show key info per columns
keywords = ['cctp', 'aapc', 'regl', 'ccp']
list_col_added = [col for col in df_consult_elevated.columns if any(keyword in col for keyword in keywords)]
mask_non_null = df_consult_elevated[list_col_added].map(lambda x: pd.notna(x) and (x != 0 or x != ""))
count_non_null_rows = mask_non_null.any(axis=1).sum()

filtered_df = df_consult_elevated[mask_non_null.any(axis=1)]
TableReport(filtered_df)

Processing column  36 / 36


,,,,,,,,,,,,,,,,


In [25]:
# Save the DataFrame to a pickle file

df_consult_elevated.to_pickle(path_df_consult_elevated)
print(f"\nDataFrame successfully saved to '{path_df_consult_elevated}'")


DataFrame successfully saved to 'C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output\df_consult_elevated.pkl'
